<a href="https://colab.research.google.com/github/rus4787/Text-processing/blob/main/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обработка текстом

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_excel("data_call_extended.xlsx")

In [ ]:
import re

def preprocess_conversation(text):
    # Приводим текст к нижнему регистру
    text = text.lower()

    # Заменяем стоп-слова
    stopwords = {
        "ну", "как бы", "типа", "это самое", "значит", "короче", "слушай", "понимаешь",
        "вообще", "на самом деле", "в общем", "просто", "кстати", "эээ", "эм", "ммм",
        "вот", "знаешь", "получается", "собственно", "в принципе", "короче говоря", "так сказать",
        "ясное дело", "ну как бы", "вообще-то", "да, ну", "наверное", "кажется", "честно говоря",
        "в любом случае", "так вот", "между прочим", "кстати говоря", "ну ладно", "точнее",
        "вот это", "прямо", "то есть"
    }
    # Удаляем стоп-слова и лишние символы, кроме меток 1) и 2)
    text = re.sub(r'\b(?:{})\b'.format('|'.join(stopwords)), '', text)
    text = re.sub(r'[^\w\s\d():]', '', text)  # Убираем лишние символы

    # Заменяем "1)" на "М:" и "2)" на "К:"
    text = text.replace("1)", "М:").replace("2)", "К:")

    # Объединяем реплики одного говорящего
    result = []
    lines = text.splitlines()  # Разбиваем текст по строкам
    current_speaker = None
    buffer = []

    for line in lines:
        # Разделяем текст на фрагменты по меткам говорящих
        parts = re.split(r'(М:|К:)', line)
        for part in parts:
            # Если найден новый говорящий
            if part in {'М:', 'К:'}:
                # Если сменился говорящий, добавляем буфер в результат
                if current_speaker and buffer and current_speaker != part:
                    result.append(f"{current_speaker} {' '.join(buffer).strip()}")
                    buffer = []
                current_speaker = part
            else:
                # Убираем лишние пробелы и добавляем текст в буфер
                cleaned_part = re.sub(r'\s+', ' ', part).strip()
                if cleaned_part:
                    buffer.append(cleaned_part)

    # Добавляем последний буфер в результат
    if current_speaker and buffer:
        result.append(f"{current_speaker} {' '.join(buffer).strip()}")

    # Собираем итоговый текст
    processed_text = ' '.join(result)

    return processed_text


In [ ]:
# Пример использования функции
text = """
1)  Алло, что-то сбросилось. По поводу... Ну как там, это же законодательно у вас должно быть.2)  Алло.2)  Да.2)  Нам не надо.2)  Девушка.2)  Вот, ну,2)  слово нет,2)  оно меняет только одно обозначение. Нет, это нет.2)  Ну, все.1)  Ну, я понимаю, что вы, наверное, не сталкивались пока что.2)  Другого обозначения у этого слова нет.2)  Нам не надо.2)  Да, девушка.2)  У меня опыт,2)  там, где вы учились, я там преподавал.2)  И мне рассказывать,2)  сталкивался, не сталкивался, не надо.
"""

In [ ]:
processed_text = preprocess_conversation(text)
print(processed_text)

М: алло чтото сбросилось по поводу как там это же законодательно у вас должно быть К: алло да нам не надо девушка слово нет оно меняет только одно обозначение нет это нет все М: я понимаю что вы не сталкивались пока что К: другого обозначения у этого слова нет нам не надо да девушка у меня опыт там где вы учились я там преподавал и мне рассказывать сталкивался не сталкивался не надо


In [ ]:
text_1 = """
2)  Библиотека. Здравствуйте. Я слушаю вас.1)  Добрый день. С Натальей Валерьевной могу поговорить. Меня зовут Елена Владимировна, институт Клавпро.1)  Работали с вашими коллегами в соседнем регионе. Они оставили ваши контакты как рекомендации,1)  обучались у нас по охране труда, успешно прошли проверку контрольных органов, звоню узнать, как у вас охрана труда.2)  Мы обучились все уже в прошлом году.1)  Скажите, у вас вообще сколько человек в штате? 130. А обученных в учебном центре сколько?1)  Вот так примерно сколько человек получается.2)  Я занимаюсь обучением.2)  Я занимаюсь обучением.1)  Ольга Владленова надо заниматься.1)  Наталья Валерьевна, вы в курсе, что у вас в охране труда каждый работник должен обучиться?1)  Протокол на него должен быть с регистрационным номером. Пристрелин труда зарегистрирован.1)  А вы говорите, кому надо было обучиться? Я так понимаю, это вы руководителей обучали?2)  Я занимаюсь обучением.1)  Всех обучали. Понятно. В этом и в прошлом году.1)  А спецоценку условий труда, оценку профессиональных рисков это делали?2)  Я занимаюсь обучением.1)  Хорошо, замечательно, что у вас все есть.1)  Наталья Валерьевна, тогда по другим вопросам у вас уточню. Вы как руководитель по экологической безопасности когда обучались?1)  Я не помню.1)  Давайте посмотрим точно, по какой программе, когда чтобы не ошибиться.2)  Вы что от меня хотите?2)  Чтоб я у вас обучилась?1)  Работали с вашими коллегами, они оставили вам номер телефона как рекомендации.2)  Ну, я понимаю, что вы сбросьте нам коммерческое или что-то.2)  Если у нас будет в чем-то потребность, мы с вами свяжемся.2)  Потому что у меня тут слишком много их всяких разных.2)  Сейчас мне не совсем удобно в них всех ковыряться.1)  По основным бы посмотрели.2)  Вы понимаете, что у меня нет на это время сейчас смотреть по основным.2)  Ну, вот по антитеррору только обучилась, допустим, по экологии.2)  Ну, по закупкам обучилась.2)  Чему я еще тут обучилась?1)  По закупкам тоже в этом году учились?2)  В прошлом училась.1)  По антитеррору в этом году?2)  По антитеррору в этом, да.1)  А помимо вас кто еще обучался по антитеррору?2)  Вы заместите меня, я ведущая.1)  Три человека получается?1)  По закупкам одна учитесь?2)  Нет, у нас есть контрактный управляющий.2)  Еще этот, господи, как он, руководитель контрактной службы.2)  Все обучены.1)  В одно время проходили обучение?2)  Да.1)  Три человека получается.1)  А по гражданской обороне не помните?2)  Мы тоже обучены.1)  Платье?2)  У нас свой центр здесь есть, Новороссийский.2)  Мы там обычно обучаемся.2)  Учебный центр у нас там бесплатно нас обучают.1)  Бесплатно даже.1)  У вас кто-то плачет?2)  Ну, наверное, кто-то обучит.1)  Отправляют вас централизованы на обучение?2)  Ну, запрашивают списки, у кого там сроки подходят, или у нас принятые заведующие обучаются.2)  У нас с сегодняшнего дня только ушла, я не знаю, когда, но через неделю, наверное.2)  Ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну, ну
"""

In [ ]:
processed_text = preprocess_conversation(text_1)
print(processed_text)

К: библиотека здравствуйте я слушаю вас М: добрый день с натальей валерьевной могу поговорить меня зовут елена владимировна институт клавпро работали с вашими коллегами в соседнем регионе они оставили ваши контакты как рекомендации обучались у нас по охране труда успешно прошли проверку контрольных органов звоню узнать как у вас охрана труда К: мы обучились все уже в прошлом году М: скажите у вас сколько человек в штате 130 а обученных в учебном центре сколько так примерно сколько человек К: я занимаюсь обучением я занимаюсь обучением М: ольга владленова надо заниматься наталья валерьевна вы в курсе что у вас в охране труда каждый работник должен обучиться протокол на него должен быть с регистрационным номером пристрелин труда зарегистрирован а вы говорите кому надо было обучиться я так понимаю это вы руководителей обучали К: я занимаюсь обучением М: всех обучали понятно в этом и в прошлом году а спецоценку условий труда оценку профессиональных рисков это делали К: я занимаюсь обучение

## Цикл обработки

In [ ]:
import pandas as pd
import time

# Создаем пустой список для хранения очищенных текстов
cleaned_texts = []

# Устанавливаем размер чанка
chunk_size = 30

# Переменные для отслеживания времени
total_time = 0  # Общее время обработки
total_chunks = 0  # Общее количество обработанных чанков
total_lines = 0  # Общее количество обработанных строк

# Время обработки каждого чанка
chunk_times = []
line_times = []

# Проходим по DataFrame по 30 строк за раз
for i in range(0, len(df), chunk_size):
    start_time = time.time()  # Засекаем время начала обработки чанка

    chunk = df['text'].iloc[i:i + chunk_size]  # Извлекаем текущий чанк (по 30 строк)

    # Применяем обработчик к каждой строке в текущем чанке и отслеживаем время
    cleaned_chunk = chunk.apply(preprocess_conversation)

    # Добавляем очищенные тексты в список
    cleaned_texts.extend(cleaned_chunk.tolist())

    # Время обработки чанка
    chunk_end_time = time.time()
    chunk_time = chunk_end_time - start_time
    chunk_times.append(chunk_time)

    # Время обработки каждой строки
    line_times.extend([chunk_time / len(chunk)] * len(chunk))

    total_time += chunk_time
    total_chunks += 1
    total_lines += len(chunk)

# Добавляем очищенные тексты в DataFrame как новый столбец 'text_clean'
df['text_clean'] = cleaned_texts

# Вычисляем среднее время обработки чанка и строки
avg_chunk_time = total_time / total_chunks if total_chunks > 0 else 0
avg_line_time = total_time / total_lines if total_lines > 0 else 0

# Выводим статистику
print(f"Общее время обработки: {total_time:.2f} секунд")
print(f"Общее количество обработанных чанков: {total_chunks}")
print(f"Общее количество обработанных строк: {total_lines}")
print(f"Среднее время обработки одного чанка: {avg_chunk_time:.5f} секунд")
print(f"Среднее время обработки одной строки: {avg_line_time:.5f} секунд")


Общее время обработки: 2.81 секунд
Общее количество обработанных чанков: 388
Общее количество обработанных строк: 11635
Среднее время обработки одного чанка: 0.00725 секунд
Среднее время обработки одной строки: 0.00024 секунд
